In [ ]:
import sympy
from sympy import Symbol, sqrt, cos, pi, symbols
import numpy as np
import matplotlib.pylab as plt

import qnet
from qnet.algebra import ScalarTimesOperator, pattern

import QDYN

from notebook_plots import plot_bs_decay, display_hamiltonian, display_eq
from single_sided_network import network_slh
from single_sided_node import node_slh
from qdyn_model import make_qdyn_oct_model

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
qnet.init_printing()

$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# Optimized Initialization of a Dicke State

## System description

### Description of a network node

Each node of our network consists of a driven $\Lambda$-type atom embedded in a cavity [@CiracPRL1997]. After adiabatic elimination of the $\ket{r}$-state, the Hamiltonian and Lindblad operator of the node read

In [ ]:
slh_node = node_slh(1, n_cavity=4)
display_hamiltonian(slh_node.H)

and

In [ ]:
display_eq('\Op{L} ', slh_node.L)

respectively.

Adding a coherent displacement feeding into the node additionally drives the cavity:

In [ ]:
Ω_α = Symbol('Omega_alpha'); κ = Symbol('kappa', positive=True)
H_node_driven = (slh_node << qnet.cc.Displace('W', alpha=Ω_α.conjugate()/sqrt(κ))).toSLH().H
H_node_driven

Here, we have renormalized the displacement amplitude $\alpha$ as $\Omega_a = \alpha^* / \sqrt{\kappa}$

It is instructive to write out this Hamiltonian in a matrix representation,

In [ ]:
display_eq('\Op{H}', qnet.convert.convert_to_sympy_matrix(H_node_driven))

where the rows/colums correspond to the following quantum numbers:

In [ ]:
slh_node.H.space.basis

This illustrates how the static interaction Hamiltonian $\Op{H}_{\text{int}}$ acts to shift a subset of levels into resonance:
* The driven sideband $\ket{g, n+1} \leftrightarrow \ket{e, n}$ via the control $\Omega_1$ is resonant only for $n=0$, but is detuned by $n \frac{g^2}{\Delta}$ for $n > 0$.
* The driven excitation of the cavity $\ket{g/e, n} \leftrightarrow \ket{g/e, n+1}$ via the control $\Omega_{\alpha}$ is resonant only for the ground state $\ket{g}$ of the qubit, but is detuned by $n \frac{g^2}{\Delta}$ when the qubit is in the excited state $\ket{e}$

### Four-node network

We now consider four of the above nodes, with feedback through a beamsplitter

In [ ]:
n_nodes = 4

In [ ]:
slh = network_slh(n_cavity=3, n_nodes=n_nodes, topology='driven_bs_fb')

The Lindblad operator of the total network corresponds to the sum of Lindblad operators of the individual nodes

In [ ]:
display_eq('\Op{L}', slh.L)

Note that this allows for constructive or destructive interference between the individual terms: The system is in a *dark state* if $\bra{\Psi} \Op{L} \ket{\Psi} = 0$

The presence of the beamsplitter results in the decay rate depending on the mixing angle $\theta$ of the beamsplitter

In [ ]:
plot_bs_decay(slh.Ls[0])

In [ ]:
θ = Symbol('theta', real=True)
slh.L.substitute({θ: 3*sympy.pi/2}).simplify_scalar()

The Hamiltonian of the entire network reads as

In [ ]:
display_hamiltonian(slh.H)

## Dimensionless Units and Choice of Parameters

All of the symbols in the Hamiltonian (including the control fields) are in units of energy. In order to work with dimensionless numbers, we express these energies in units of $g$. This implies time units of $\hbar/g$.

The controls are

In [ ]:
control_syms = sorted([sym for sym in slh.H.all_symbols() if sym.name.startswith('Omega')], key=str)
control_syms

This leaves the remaining free symbols

In [ ]:
syms = slh.all_symbols().difference(control_syms);
syms

which we set as follows, with all nodes having identical parameters

In [ ]:
Delta =   100.0
g     =     1.0 # by definition
kappa =     0.01
E0    =     1.4
theta = 0.0 * np.pi
num_vals = {
    Symbol('kappa', positive=True): kappa,
    Symbol('theta', real=True):     theta,
}
for i_node in range(n_nodes):
    num_vals[Symbol('Delta_%d' % (i_node + 1), real=True)] = Delta
    num_vals[Symbol('g_%d' % (i_node + 1), positive=True)] = g
num_vals

In total, this leads to the following scalar coefficients in the Hamiltonian

In [ ]:
{str(t.coeff): t.coeff.subs(num_vals) for t in pattern(head=ScalarTimesOperator).findall(slh.H)}

## Overview of the Protocol

Note that except for $\Op{H}_{\alpha}$, the Hamiltonian preserves excitations. We therefore propose a two-step protocol for the initialization of a Dicke state. Conceptually, for time $t < 0$, we used the cavity drive and local operation to excite the first $N/2$ qubits. Then, for $t > 0$, we exploit the channel-meditated interaction between the nodes to generate the entangled Dicke state, while keeping the system in a dark state in order to suppress dissipation.

The initialization step can be performed through the following steps:
* Excite the cavity on each node by driving $\Op{H}_{\alpha}$
* On the first $N/2$ nodes, drive the sideband transition to transfer the excitation to the qubit
* Wait for the excitation in the second $N/2$ nodes to decay to the ground state

The total scheme is defined on a time grid as follows:

In [ ]:
tgrid_start = -1000
tgrid_end =  1000
nt = tgrid_end - tgrid_start + 1
tgrid = QDYN.pulse.pulse_tgrid(t0=tgrid_start, T=tgrid_end, nt=nt)

## Implementation of the initialization step

We build a guess for a initialization scheme on a series of Blackman pulses, where each pulse acting for a duration of $T$ with peak amplitude $E_0$ has the shape

In [ ]:
a, b, E0, t, T, μ, t_π = sympy.symbols("a b E_0, t, T, mu, t_pi", positive=True)
B_form = (E0/2) * (1 - a - cos(2 * pi * t/T) + a * cos(4 * pi* t/T))
display_eq('B(t)', B_form)

with $a = 0.16$. This shape looks almost identical to a Gaussian with covering a $\pm 3\sigma$ interval in the window $T$, but unlike the Gaussian, the Blackman shape is exactly zero at 0 and $T$.

In [ ]:
a_blackman = 0.16

If we assume that the pulse appears in the Hamiltonian with a prefactor $\mu$, the effective pulse aread covered per unit $T$ is 

In [ ]:
Ωeff_form = μ * (sympy.integrate(B_form, (t, 0, T)) / T).simplify()
display_eq(r'\Omega_{\text{eff}}', Ωeff_form)

In a two-level system (with zero detuning), this will lead to a population inversion in the time

In [ ]:
t_pi_form = pi / (2 * Ωeff_form).subs({1-a: b}) # protect 1-a, so sympy doesn't do weird signs
display_eq(t_π, t_pi_form.subs({b: 1-a}))

Alternatively, for a fixed time $t_\pi$, we have to choose the amplitude as

In [ ]:
E_pi_form = sympy.solve(t_pi_form - t_π, E0)[0].subs({b: 1-a})
display_eq('E_\pi', E_pi_form)

In [ ]:
def pi_pulse(tgrid, t_start, t_stop, mu):
    E0 = sympy.N(E_pi_form.subs({t_π: t_stop-t_start, μ: mu, a: a_blackman}))
    pulse = QDYN.pulse.Pulse(
        tgrid, amplitude=(E0 * QDYN.pulse.blackman(tgrid, t_start, t_stop)),
        time_unit='dimensionless', ampl_unit='dimensionless', freq_unit='dimensionless')
    return pulse

In [ ]:
def initialization_controls():
    controls = {
        Symbol('Omega_alpha'): pi_pulse(tgrid, t_start=-200, t_stop=-100, mu=0.5*np.sqrt(2)),
    }
    pulse_qubit = pi_pulse(tgrid, t_start=-100, t_stop=0, mu=1.0)
    for ctrl_sym in [Symbol('Omega_%d' % ind) for ind in range(1, n_nodes+1)]:
        controls[ctrl_sym] = pulse_qubit
    return controls

In [ ]:
initialization_model = make_qdyn_oct_model(
    slh, num_vals, initialization_controls(), energy_unit='dimensionless',
    mcwf=True, non_herm=True, oct_target='dicke',
    lambda_a=1e-4)

## Example optimization commands

*   hilbert space optimization

    ```OMP_NUM_THREADS=1 qdyn_optimize --internal-units=GHz_units.txt --debug --J_T=J_T_re .```

*   trajectory optimization (model with `mcwf=True`)

    ```OMP_NUM_THREADS=1 qdyn_optimize --internal-units=GHz_units.txt --n-trajs=20 --J_T=J_T_re .```